In [4]:
%load_ext autoreload
%autoreload 2

In [78]:
import openai
from openai_utils import *
import pandas as pd
import sys
sys.path.append('../../sft')
sys.path.append('../../')
from utils import get_perc_first_correct, calc_ncdg
from train_utils import remove_ranking, process_ranking, get_labels
from tqdm import tqdm
import numpy as np

In [8]:
client = get_client()

GPT-4, Original Prompt

In [29]:
df = pd.read_csv('../../data/split/test.csv')
results = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    # TODO: add error handling / timeout
    prompt, _ = remove_ranking(row.text)#(row.text.split('## Ranking: ')[0] + "## Ranking: ")
    results.append(
       get_openai_response(prompt, client) 
    )

  0%|          | 0/525 [00:00<?, ?it/s]

100%|██████████| 525/525 [09:41<00:00,  1.11s/it]


In [39]:
process_ranking(remove_ranking(row.text)[1])

[3, 5, 1, 4, 2]

In [60]:
# Process
rankings = []
for r in results:
    res = process_ranking(r)
    assert len(res) <= 5 and len(res) >= 3
    if len(res) < 5:
        missing = list(set(np.arange(1,6)) - set(res))
        res += missing
    assert len(res) == 5
    rankings.append(res)
df['rankings'] = rankings

In [43]:
df['y_true'] = get_labels(df)
df

,text,rankings,y_true
0,"Given a query and five passages, rank the pass...","[4, 1, 2, 3, 5]","[2, 5, 3, 1, 4]"
1,"Given a query and five passages, rank the pass...","[4, 2, 1, 3, 5]","[4, 2, 3, 1, 5]"
2,"Given a query and five passages, rank the pass...","[2, 1, 3, 4, 5]","[2, 1, 4, 5, 3]"
3,"Given a query and five passages, rank the pass...","[1, 4, 5, 2, 3]","[1, 2, 3, 4, 5]"
4,"Given a query and five passages, rank the pass...","[2, 1, 4, 5, 3]","[1, 3, 4, 2, 5]"
...,...,...,...
520,"Given a query and five passages, rank the pass...","[3, 5, 4, 1, 2]","[3, 5, 1, 2, 4]"
521,"Given a query and five passages, rank the pass...","[5, 1, 3, 2, 4]","[1, 4, 5, 2, 3]"
522,"Given a query and five passages, rank the pass...","[2, 5, 1, 4, 3]","[3, 2, 4, 5, 1]"
523,"Given a query and five passages, rank the pass...","[1, 2, 3, 5, 4]","[3, 2, 1, 5, 4]"


In [61]:
get_perc_first_correct(df['y_true'].to_list(), rankings)

0.2742857142857143

In [79]:
calc_ncdg(df['y_true'].to_list(), rankings)

0.7749093764049221